# Materials Project：APIの利用法 (2023/2)

2022年にリニューアルされたMaterials Project のデータをAPIで一括DLなどするためのスクリプトです。<br>
基本的にdocumentに書いてあるtutorial以上のことはしていません。<br>
<strong> inset-APIkey-here </strong>と書かれた場所に自身のAPIキーを入力してください。<br> 
出力ファイルがあれば、現在のディレクトリーにある <strong>output</strong> ディレクトリーに格納 <br>

勘所が分かれば、下記のexample集を活用すると便利<br>
https://docs.materialsproject.org/downloading-data/using-the-api/examples#band-gaps-for-all-materials-containing-only-si-and-o<br><br>

＊2025/12  Google colab使用者用に修正

<div style="text-align: right;">
2023年02月05日作成 中山<br>
2025年12月02日補修 中山<br>

<br>
</div>

### APIを使うためのライブラリーインストール（動作しない場合、# を外して使ってください。)

In [ ]:
##　強制インストール(Google colabの場合)
# !pip install --force-reinstall --no-cache-dir \
#     mp_api==0.45.13 \
#     emmet-core==0.86.1 \
#     pymatgen

### API key の取得とMPResterのインポート（必須）

<strong> inset-APIkey-here </strong>と書かれた場所に自身のAPIキーを入力してください。<br> 
keyは https://materialsproject.org/api#documentation から参照できます(2023/2/5現在)。<br>


In [ ]:
import os
#api_key=os.environ['MPIDN']　環境変数 MPIDNから取得する場合
#api_key="ir30n**********"  入力例　
api_key="--insert your API key here--"

from mp_api.client import MPRester

#outputフォルダーの作成
if not os.path.isdir('output_mp'):
    os.mkdir('output_mp')

### 例　Li金属(bcc構造) 登録番号 mp-135  の結晶構造ファイルをダウンロードする   

In [ ]:
with MPRester(api_key) as mpr:

    docs = mpr.summary.search(material_ids=["mp-135"], fields=["structure"])
      #docsは複数の材料を格納するための変数　docs[0], docs[1], docs[2], .... の順にサンプル毎のデータが格納される
      #materials_idがmp-135となっているデータを取得する
      #fields=**  取得するデータの対象（structure 結晶構造データ
    structure = docs[0].structure
      #今回は1つのサンプルデータだけを取り出したのでdocs[0]にデータが格納されている。
      # オブジェクト .structureで構造データを参照 -->  structureという変数に格納
    print (structure)
      #とりあえず structure という変数に入ったデータを閲覧
    structure.to(filename="output/mp-135.cif")
    structure.to(filename="output/POSCAR.mp-135")
    

### 取得できるデータの確認

In [ ]:

with MPRester(api_key) as mpr:
    list_of_available_fields = mpr.summary.available_fields

list_of_available_fields


### 複数のプロパティーを出力する（画面出力）

In [ ]:
with MPRester(api_key) as mpr:


    #データのダウンロード  （検索キー　material_ids=****,  取得データ fields=**** )
    docs = mpr.summary.search(material_ids=["mp-10499"], fields=["material_id","composition","formula_pretty","chemsys","energy_above_hull"])

    
    #取得データをprint文で閲覧
    print ("material_id",docs[0].material_id)
    print ("composition",docs[0].composition)
    print ("formula_pretty",docs[0].formula_pretty)
    print ("chemsys",docs[0].chemsys)
    print ("energy_above_hull",docs[0].energy_above_hull)


### 組成範囲を指定して複数のデータを一括して入手する　（パターン１： chemsysを使う）

  chemsys="Li-Co-O":  Li, Co, Oのすべてを必ず含み、それ以外の元素は出力させない ( CoO2 や Li2CrCoO4　などは対象外となる)<br><br>


In [ ]:
with MPRester(api_key) as mpr:
    docs = mpr.summary.search(chemsys="Li-Co-O", fields=["material_id","formula_pretty","structure","energy_above_hull"])
    
    f = open ('output/material1.dat','w')  #出力するファイル名

    #データのファイル出力  docsにある複数データを docとして取り出す
    for doc in docs:
        mpid = doc.material_id
        doc.structure.to(filename="output/"+mpid+doc.formula_pretty+".cif")
        f.write(mpid+','+doc.formula_pretty+','+str(doc.energy_above_hull)+'\n') #strは数字などのデータを文字列扱いに変換する関数
    f.close()
       
        

### 組成範囲を指定して複数のデータを一括して入手する　（パターン２： elementsを使う）

  elements=["Li","Co","O"]:  Li, Co, Oのすべてを必ず含めば、他の元素が入っていても構わない ( CoO2は対象外だが Li2CrCoO4はデータがあれば取得する)<br><br>


In [ ]:
with MPRester(api_key) as mpr:
    docs = mpr.summary.search(elements=["Li","Co","O"], fields=["material_id","formula_pretty","structure","energy_above_hull"])
    
    f = open ('output/material2.dat','w')  #出力するファイル名

    #データのファイル出力  docsにある複数データを docとして取り出す
    for doc in docs:
        mpid = doc.material_id
        doc.structure.to(filename="output/"+mpid+doc.formula_pretty+".cif")
        f.write(mpid+','+doc.formula_pretty+','+str(doc.energy_above_hull)+'\n') #strは数字などのデータを文字列扱いに変換する関数
    f.close()

## Google colabを使っている場合　
以下のコマンドを実行して、ファイルをダウンロードできます

In [ ]:
import shutil
from google.colab import files
import os

# outputフォルダをzip圧縮
zip_filename = 'output_files'
output_directory = 'output_mp'
shutil.make_archive(zip_filename, 'zip', output_directory)

# 生成されたzipファイルをダウンロード
files.download(zip_filename + '.zip')